In [1]:
import pennylane as qml
import cv2
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer
import glob
from tqdm import tqdm
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

In [3]:
np.random.seed(420)

In [4]:
#Lets load in our training data. We got this data from the Pytorch CNN tutorial. It is images of dogs and cats grayscale with labels accordingly
training_data = np.load("training_data.npy", allow_pickle=True)
#print(training_data)

In [5]:
#Lets separate out the images and the labels
images = []
labels = [] 

for i in training_data:
    images.append(i[0])
    labels.append(i[1])
#print(images[1:10])
#print(labels)

In [6]:
#Split data into train and test datasets
train_images = images[0:1000]
test_images = images[1001:1200]

train_labels = labels[0:1000]
test_labels = labels[1001:1200]

In [7]:
def density_matrix(state):
    """Calculates the density matrix representation of a state.

    Args:
        state (array[complex]): array representing a quantum state vector

    Returns:
        dm: (array[complex]): array representing the density matrix
    """
    return state * np.conj(state).T

In [8]:
label_0 = [[1], [0]]
label_1 = [[0], [1]]
state_labels = [label_0, label_1]

In [9]:
dev = qml.device("default.qubit", wires=1)

In [10]:
@qml.qnode(dev)
def qcircuit(params, x=None, y=None):
    """A variational quantum circuit representing the Universal classifier.

    Args:
        params (array[float]): array of parameters
        x (array[float]): single input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    for p in params:
        qml.Rot(*x, wires=0)
        qml.Rot(*p, wires=0)
    return qml.expval(qml.Hermitian(y, wires=[0]))

In [11]:
def cost(params, x, y, state_labels=None):
    """Cost function to be minimized.

    Args:
        params (array[float]): array of parameters
        x (array[float]): 2-d array of input vectors
        y (array[float]): 1-d array of targets
        state_labels (array[float]): array of state representations for labels

    Returns:
        float: loss value to be minimized
    """
    # Compute prediction for each input in data batch
    loss = 0.0
    dm_labels = [density_matrix(s) for s in state_labels]
    if np.argmax(y) == 1:
        y_use = dm_labels[1]
    elif np.argmax(y) == 0:
        y_use = dm_labels[0]
    for i in range(len(x)):
        for j in range(len(x[i])):
            f = qcircuit(params, x=[x[i][j][0],x[i][j][1],x[i][j][2]], y=y_use)
            loss = loss + (1 - f) ** 2
    return loss / (len(x)*len(x[i]))

In [12]:
def test(params, x, y, state_labels=None):

    #Tests on a given set of data.

    #Args:
    #    params (array[float]): array of parameters
    #    x (array[float]): 2-d array of input vectors
    #    y (array[float]): 1-d array of targets
    #    state_labels (array[float]): 1-d array of state representations for labels

    #Returns:
    #    predicted (array([int]): predicted labels for test data
    #    output_states (array[float]): output quantum states from the circuit

    fidelity_values = []
    dm_labels = [density_matrix(s) for s in state_labels]
    predicted = []

    for i in range(len(x)):
        for j in range(len(x[i])):
            fidel_function = lambda y: qcircuit(params, x=[x[i][j][0],x[i][j][1],x[i][j][2]], y=y)
            fidelities = [fidel_function(dm) for dm in dm_labels]
            best_fidel = np.argmax(fidelities)

        predicted.append(best_fidel)
        fidelity_values.append(fidelities)

    return np.array(pd.get_dummies(predicted)), np.array(fidelity_values)

In [13]:
def accuracy_score(y_true, y_pred):
    """Accuracy score.

    Args:
        y_true (array[float]): 1-d array of targets
        y_predicted (array[float]): 1-d array of predictions
        state_labels (array[float]): 1-d array of state representations for labels

    Returns:
        score (float): the fraction of correctly classified samples
    """
    score = 0
    print(y_pred)
    print(y_true)
    for i in range(len(y_true)):
        if np.argmax(y_true[i]) == np.argmax(y_pred[i]):
            score += 1 
    print(score/len(y_true))
    return (score / len(y_true))

In [14]:
def iterate_minibatches(inputs, targets, batch_size):
    """
    A generator for batches of the input data

    Args:
        inputs (array[float]): input data
        targets (array[float]): targets

    Returns:
        inputs (array[float]): one batch of input data of length `batch_size`
        targets (array[float]): one batch of targets of length `batch_size`
    """
    for start_idx in range(0, inputs.shape[0] - batch_size + 1, batch_size):
        idxs = slice(start_idx, start_idx + batch_size)
        yield inputs[idxs], targets[idxs]

In [15]:
# Generate training and test data
num_training = 200
num_test = 2000

# Train using Adam optimizer and evaluate the classifier
num_layers = 3
learning_rate = 0.1
epochs = 10
batch_size = 32

opt = AdamOptimizer(learning_rate, beta1=0.9, beta2=0.999)

# initialize random weights
params = np.random.uniform(size=(num_layers, 3))

predicted_train, fidel_train = test(params, np.array(train_images), np.array(train_labels), state_labels)
accuracy_train = accuracy_score(np.array(train_labels), predicted_train)

predicted_test, fidel_test = test(params, np.array(test_images), np.array(test_labels), state_labels)
accuracy_test = accuracy_score(np.array(test_labels), predicted_test)

# save predictions with random weights for comparison
initial_predictions = predicted_test

loss = cost(params, np.array(train_images), np.array(train_labels), state_labels)

print(
    "Epoch: {:2d} | Cost: {:3f} | Train accuracy: {:3f} | Test Accuracy: {:3f}".format(
        0, loss, accuracy_train, accuracy_test
    )
)

for it in range(epochs):
    for Xbatch, ybatch in tqdm(iterate_minibatches(np.array(train_images), np.array(train_labels), batch_size=batch_size)):
        params = opt.step(lambda v: cost(v, Xbatch, ybatch, state_labels), params)

    predicted_train, fidel_train = test(params, np.array(train_images), np.array(train_labels), state_labels)
    accuracy_train = accuracy_score(np.array(train_labels), predicted_train)
    loss = cost(params, np.array(train_images), np.array(train_labels), state_labels)

    predicted_test, fidel_test = test(params, np.array(test_images), np.array(test_labels), state_labels)
    accuracy_test = accuracy_score(np.array(test_labels), predicted_test)
    res = [it + 1, loss, accuracy_train, accuracy_test]
    print(
        "Epoch: {:2d} | Loss: {:3f} | Train accuracy: {:3f} | Test accuracy: {:3f}".format(
            *res
        )
    )
    print(predicted_test)

KeyboardInterrupt: 